# Exective summary of Work Package 2

## Objectives

This work package aims at developing a data-driven fault detection algorithm for the motors in the robot. The algorithm should be able to accurately detect faults under different working conditions of the robot, and to display the results in the GUI developed in WP1.

## Tasks

- Task 1: Training data generation: Generate training data for your model, considering different working conditions of the robot.
- Task 2: Model development.
  - Data cleaning, preprocessing
  - Investigate different possible model and train the final model
- Task 3: GUI design and implementation.
- Task 4: Data challenge: Each group will generate a separate dataset, which will be used to construct a final evaluation dataset for the other groups. The performance of your model will be evaluated through this testing dataset.

## Delierables

- A Jupyter notebook reporting the model development process, including but not limited to:
  - Data cleaning, preprocessing
  - The models you tried and their performance evaluation through cross validation
  - The final model with best performance
- A demo software for condition-monitoring and fault detection:
  - The software should be able to read the data from the robot and display the results in the GUI.
  - The software should be able to detect faults in the robot and display the results in the GUI.
  - Record a video with the robot to demonstrate the functionality of your software.


# Working plan for today's TD

Today, we mainly work on task 2. More specifically, we will work on the following:
1. Explore the data we collected last week, including:
    - Data visualization to explore the patterns of the data
    - Use Principle Component Analysis (PCA) or t-SNE to visualize the data, in order to have a first flavor about the difficulty of the problem 
    - Explore if we have a significant difference between different sequences through visualization and clustering
2. Data cleaning and preprocessing
    - Missing values and outlier handling
    - Feature engineering
3. Apply statistical testing approach.
4. Apply a simple logistic regression model as a benchmark model for the classification task.

Before starting, please:
- Fetch the most up-to-date version of the github repository.
- Create a new branch called "WP2_TD_Lect_6_YourName", based on the branch "zhiguo_dev_DARR" and switch to it.
- Copy the notebook WP2_DATA-DRIVEN FAULT DETECTION/support/WP2_TD_Lect_6 to WP2_DATA-DRIVEN FAULT DETECTION/TD_6/, and rename it to TD_Lect_6_YourName.ipynb
- After finishing this task, push your changes to your github repository.
- Submit a pull request to the "zhiguo_dev" branch.

# Exercise 5 Implement logistic regression

Logistic regression is a simple and powerful machine learning algorithm. It is used for classification problems. It is a binary classifier, meaning that it can distinguish between only two classes. Often, it is used as a "quick and dirty" method to create benchmark for a classification problem.

In this exercise, we will implement logistic regression in our dataset. You will try:
- Use sklearn to implement logistic regression
- Hyper-parameter tuning in logistic regression
- Use cross-validation to evaluate the performance of your model



### Task 1: Effect of hyper-parameter and normalization

Below is a sample code for logistic regression.
- Try to change different values of hyper-parameter $C$, and see the impact on the results.
- In this code, we did not do any preprocessing on the data. You can try to implement your preprocessing and see the impacts on the results.

In [25]:
from utility import read_data

# Root path to the CSV files
path_normal = r'../Data collection_20231109/Normal sequence'
path_failure = r'../Data collection_20231109/Failure sequence'
df = read_data(path_normal, path_failure)

In [26]:
ptv_features = [col for col in df.columns if col.endswith(('position', 'temperature', 'voltage'))]
window_median = 10
window_mean = 20

df_smoothed = df.copy()

# Group the dataframe by sequence_idx and apply smoothing to each group separately
for seq_idx, group in df_smoothed.groupby('sequence_idx'):
    for feature in ptv_features:
        group[feature] = group[feature].rolling(window=window_median, min_periods=1).median()
        group[feature] = group[feature].rolling(window=window_mean, min_periods=1).mean()
    df_smoothed.loc[group.index] = group

In [27]:
# Separate the features (X) and the target variable (y)
X = df_smoothed.drop(['label', 'sequence_idx', 'time'], axis=1)
y = df_smoothed['label']

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# Train a logistic regression model
logreg = LogisticRegression(C=0.1)
logreg.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = logreg.predict(X_test)

# Compute and print the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Calculate recall
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

[[337  61]
 [ 53 268]]
Accuracy: 0.8414464534075105
Precision: 0.8145896656534954
Recall: 0.8348909657320872


## Task 2: Use $k$-fold cross validation to evaulate the performance

In the previous sample code, we used a single train-test split to evaluate the performance of our model. However, in practice, we want to use a more robust method to evaluate the performance of our model. One common method is to use $k$-fold cross validation. In this task, we will use $k$-fold cross validation to evaluate the performance of our model. Please implement a cross-validation with $k=5$, and calculate the average accuracy, precision, recall and F1 score of the tests.

This is a routine task which can be easily sovled by current LLM like chatgpt or Tongyi Lingma. I let you try to generate your code by yourself.



In [37]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# Define the k-fold cross validation
k = 5

# Create a logistic regression model
logreg = LogisticRegression(C=0.1)

# Print the average accuracy, precision, recall and F1 score of the tests
print("Average accuracy:", cross_val_score(logreg, X, y, cv=k, scoring='accuracy').mean())
print("Average precision:", cross_val_score(logreg, X, y, cv=k, scoring='precision_macro').mean())
print("Average recall:", cross_val_score(logreg, X, y, cv=k, scoring='recall_macro').mean())
print("Average F1 score:", cross_val_score(logreg, X, y, cv=k, scoring='f1_macro').mean())


Average accuracy: 0.7780250347705147
Average precision: 0.7946751584859657
Average recall: 0.7788415630620642
Average F1 score: 0.7746116140751236


## Task 3: Tuning the hyper-parameter of the model

We can tune the hyper-parameters of our model by using the GridSearchCV function. A more efficient way is to create a pipeline that includes the hyper-parameter tuning, and prediction. You can try to generate the code using the following prompt:

```Prompt
Create a pipeline, where the first step is to use GridSearchcv to tune the hyper-parameter C of a logistic regression, and the second step is to use the best parameters to make predictions. 

Try to implement the generated code on our dataset here:

## Task 4: Use the pipeline in cross validation

Then, you can use your defined pipeline to automatically tune the hyper-parameter and use the best model for classification. Please try this on the code of Task 2, i.e., use the pipeline in the cross validation. You just need to replace the logistic classifier with the pipeline.

In [40]:
import warnings
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Ignore warnings
warnings.filterwarnings('ignore')

# Define the pipeline
pipeline = Pipeline([
    ('logreg', LogisticRegression())
])

# Define the hyper-parameters to tune
parameters = {
    'logreg__C': np.logspace(np.log10(0.01), np.log10(100), num=100),
    'logreg__penalty': ['l1', 'l2']
}

# Use GridSearchCV to tune the hyper-parameters and use the best model for classification
grid_search = GridSearchCV(pipeline, parameters, cv=5)
grid_search.fit(X, y)

# Print the best hyper-parameters
print("Best hyper-parameters:", grid_search.best_params_)

# Use the best model to make predictions and evaluate the performance using cross-validation
best_model = grid_search.best_estimator_

# Print the average accuracy, precision, recall and F1 score of the tests
print("Average accuracy:", cross_val_score(best_model, X, y, cv=5, scoring='accuracy').mean())
print("Average precision:", cross_val_score(best_model, X, y, cv=5, scoring='precision_macro').mean())
print("Average recall:", cross_val_score(best_model, X, y, cv=5, scoring='recall_macro').mean())
print("Average F1 score:", cross_val_score(best_model, X, y, cv=5, scoring='f1_macro').mean())
print("Average AUC score:", cross_val_score(best_model, X, y, cv=5, scoring='roc_auc').mean())

Best hyper-parameters: {'logreg__C': 0.7924828983539177, 'logreg__penalty': 'l2'}
Average accuracy: 0.7977746870653686
Average precision: 0.8034439404762423
Average recall: 0.7976034898415187
Average F1 score: 0.795627959871716
Average AUC score: 0.8409352030295019
